In [ ]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, roc_curve, auc


from regression_module import *
import warnings

warnings.filterwarnings('ignore')

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# Load Data Set
df = pd.read_csv('data/final_df.csv')

In [ ]:
# Remove first two unnecessary columns from DF
df = df.iloc[:,2:]

In [ ]:
# Assign target variable
y = df.churn
# Drop target variable from independent features DF
X = df.drop('churn', axis = 1)
# Save columns as list of strings for reassign after scaling
cols = X.columns
scaled_df.to_csv('data/processed_df.csv')

In [ ]:
# Instantiate a scaling object from SKlearn
mm = MinMaxScaler()
# Fit_Transform the independent features DF to the min-max scaler
scaled_X = mm.fit_transform(X)

In [ ]:
# Assign scaled dataset to pandas dataframe
scaled_df = pd.DataFrame(scaled_X)
# Reassign columns names to new dataframe
scaled_df.columns = cols

In [ ]:
# Perform a train test split, maintaining test size sample and random state from logistic regression notebook
X_train, X_test, y_train, y_test = train_test_split(scaled_df, y, test_size = .25, random_state = 33)

# Baseline KNN Classifier (default parameters)

In [ ]:
# Instantiate KNN object
knn1 = KNeighborsClassifier()

# Fit training set to our classifying object
knn1.fit(X_train, y_train)

In [ ]:
# Assign predictions to variable
test_preds1 = knn1.predict(X_test)

# Calculate (accuracy) score metric
knn1.score(X_test, y_test)

In [ ]:
# Calculate and print all four major metrics
print(f"Precision Score: {precision_score(y_test, test_preds1)}")
print(f"Recall Score: {recall_score(y_test, test_preds1)}")
print(f"Accuracy Score: {accuracy_score(y_test, test_preds1)}")
print(f"F1 Score: {f1_score(y_test, test_preds1)}")

In [ ]:
# Assign false positive rate, true pos rate, and thresholds to variables using sklearn.metrics library
fpr, tpr, threshold = roc_curve(y_test, test_preds1)
# Calculate AUC score from sklearn.metrics library
roc_auc = auc(fpr, tpr)
# Print auc score
print(f'AUC Score: {roc_auc}')

# Plot AUC curve
plt.style.use('ggplot')
plt.figure(figsize = (10,8))
plt.plot(fpr, tpr, lw = 2, label = 'Baseline AUC ='+str(roc_auc))
plt.plot([0,1],[0,1], linestyle = '--', lw = 2)
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.xlabel('False Positive Rate', fontsize = 20, fontweight = 'bold')
plt.ylabel('True Positive Rate', fontsize = 20, fontweight = 'bold')
plt.title('ROC Curve: KNN Classifier (k = 5)', fontsize = 25, fontweight = 'bold')
plt.legend(loc = 4, fontsize = 15)
plt.tight_layout()

# Cross Val - KNN Classifier

In [ ]:
# Instantiate KNN classifier object again, assigning a different name
knn_clf = KNeighborsClassifier()

# perform a cross validation score using sklearn.model_selection (iterate until maximum output score)
knn_cv_score = cross_val_score(knn_clf, X_train, y_train, cv = 4)

# Use numpy to obtain mean accuracy score from cross-validation folds and display
mean_knn_cv_score = np.mean(knn_cv_score)
print(f"Mean Cross Validation Score: {mean_knn_cv_score :.2%}")

In [ ]:
# Instantiate a second KNN classifier object
knn2 = KNeighborsClassifier()
# Establish parameters grid in dictionary form per KNN documentation
param_grid = {'n_neighbors':np.arange(1,20)}
# Pass instantiated KNN object, parameter grid, and optimal fold value as arguments
knn2_gscv = GridSearchCV(knn2, param_grid, cv = 4)
# Fit train data to our new object
knn2_gscv.fit(X_train, y_train)

In [ ]:
# Return best parameters and score
print(f"Best Parameters: {knn2_gscv.best_params_}")
print(f"Best Score: {knn2_gscv.best_score_}")

In [ ]:
# Predict test values
test_preds2 = knn2_gscv.predict(X_test)

In [ ]:
# Obtain second accuracy score of optimal model
knn2_gscv.score(X_test,y_test)

In [ ]:
# Print 4 major metrics
print(f"Precision Score: {precision_score(y_test, test_preds2)}")
print(f"Recall Score: {recall_score(y_test, test_preds2)}")
print(f"Accuracy Score: {accuracy_score(y_test, test_preds2)}")
print(f"F1 Score: {f1_score(y_test, test_preds2)}")

In [ ]:
# Obtain major metrics and plot second ROC curve
fpr, tpr, threshold = roc_curve(y_test, test_preds2)
roc_auc = auc(fpr, tpr)
print(f'AUC Score: {roc_auc}')

plt.style.use('ggplot')
plt.figure(figsize = (10,8))
plt.plot(fpr, tpr,lw = 2, label = 'KNN(k=5) AUC = '+str(roc_auc))
plt.plot([0,1],[0,1], linestyle = '--', lw = 2)
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.xlabel('False Positive Rate', fontsize = 20, fontweight = 'bold')
plt.ylabel('True Positive Rate', fontsize = 20, fontweight = 'bold')
plt.title('ROC Curve: KNN Classifier (k = 5)', fontsize = 25, fontweight = 'bold')
plt.legend(loc = 4, fontsize = 15)
plt.tight_layout()